In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# constants defintions
UPPER = 10
LOWER = -5
NUM_ITERATIONS = 100
d = 5
n = 20

In [3]:
def generate_data(seed_num, d, n):
    
    rng = np.random.default_rng(seed = seed_num)

    data = (UPPER - LOWER)* rng.random((n,d)) + LOWER # "uniformly randomly, obeying the bounds"

    return pd.DataFrame(data)

def random_parameters(seed_num, num):
    rng = np.random.default_rng(seed = seed_num)
    return rng.random((num, 2)) #uniform random, two for each d

def f(x):
    return np.square(np.sum(np.square(x)))

def best(data):
    return data[np.argmin(f(data))]

def worst(data):
    return data[np.argmax(f(data))]
    
def new_vector(current, best_i, worst_i, r1, r2):

    new_i = current + r1*(best_i - abs(current)) - r2*(worst_i - abs(current))
    
    if (f(new_i) <= f(current)):
        if (new_i < LOWER):
            return LOWER
        elif (new_i > UPPER):
            return UPPER
        else:
            return new_i
    else:
        return current

def algo(df, num_iterations):
    data = df.T # its way more intutive to iterate through the rows so transpose the dataframe
    
    updates = 0
    while (updates < num_iterations):
        r = random_parameters(42, data.shape[0])
        best_v = best(data)
        worst_v = worst(data)
        
        for i, vector in enumerate(data): #updating every vector in data
            vfunc = np.vectorize(new_vector)
            data[i] = vfunc(vector, best_v, worst_v, r.T[0], r.T[1]) 
            
        updates += 1
        
        #if any hit the global minimum, stop
        if ((data == 0).any().any()):
            break
    
    return data.T, updates

df = generate_data(42, d, n)
df

,0,1,2,3,4
0,6.609341,1.583177,7.878969,5.460520,-3.587340
1,9.634335,6.417096,6.790965,-3.078296,1.755789
2,0.561970,8.901475,4.657977,7.341424,1.651213
3,-1.591419,3.318772,-4.042741,7.414468,4.474966
4,6.371316,0.317890,9.560470,8.396817,6.675752
5,-2.080419,2.000815,-4.342944,-2.685658,5.245734
6,6.171432,9.512646,-0.112620,0.556896,2.043337
7,-2.157930,-3.051177,2.135574,-1.596360,5.047210
8,1.557279,7.490173,5.503977,-0.314500,7.483897
9,7.071465,0.812176,-0.675078,5.237433,-2.903713


In [4]:
'''plt.close("all")
df2 = df

vfunc = np.vectorize(f)
df2['f(x)'] = vfunc(df2)
print(df2)
ts = pd.DataFrame(df2)
ts.plot.scatter(x=0, y='f(x)');'''

'plt.close("all")\ndf2 = df\n\nvfunc = np.vectorize(f)\ndf2[\'f(x)\'] = vfunc(df2)\nprint(df2)\nts = pd.DataFrame(df2)\nts.plot.scatter(x=0, y=\'f(x)\');'

In [5]:
df, iters = algo(df, NUM_ITERATIONS)
df

,0,1,2,3,4
0,0.000000,0.0,0.000000,0.000000,0.000000
1,1.000000,1.0,1.000000,1.000000,1.000000
2,2.000000,2.0,2.000000,2.000000,0.000000
3,-2.903003,3.0,3.000000,-2.981207,0.899001
4,-2.238081,4.0,-1.642248,-1.956282,2.221273
5,-1.573158,5.0,0.239197,-0.931358,3.543545
6,-0.908236,6.0,2.120642,0.093567,4.865818
7,-0.243314,7.0,4.002087,1.118491,6.188090
8,0.421609,8.0,5.883532,2.143416,7.510362
9,1.086531,9.0,7.764977,3.168341,8.832635


In [6]:
'''plt.close("all")
df2 = df

vfunc = np.vectorize(f)
df2['f(x)'] = vfunc(df2)
print(df2)
ts = pd.DataFrame(df2)
ts.plot.scatter(x=0, y='f(x)');'''

'plt.close("all")\ndf2 = df\n\nvfunc = np.vectorize(f)\ndf2[\'f(x)\'] = vfunc(df2)\nprint(df2)\nts = pd.DataFrame(df2)\nts.plot.scatter(x=0, y=\'f(x)\');'

In [7]:
print("1. Population size:", n)
print("2. Numerical values of d that cause the minium value of the function:", np.array(df.T[0]))
print("3. Corresponding Minimum value of the function: ", f(df.T[0]))
print("4. Max iterations used: ", iters)
print("5. First five frames:")
print(df.head()) #first 5 frames
print("   Last Five frames:")
print(df.tail())#last 5 frames 

1. Population size: 20
2. Numerical values of d that cause the minium value of the function: [0. 0. 0. 0. 0.]
3. Corresponding Minimum value of the function:  0.0
4. Max iterations used:  1
5. First five frames:
          0    1         2         3         4
0  0.000000  0.0  0.000000  0.000000  0.000000
1  1.000000  1.0  1.000000  1.000000  1.000000
2  2.000000  2.0  2.000000  2.000000  0.000000
3 -2.903003  3.0  3.000000 -2.981207  0.899001
4 -2.238081  4.0 -1.642248 -1.956282  2.221273
   Last Five frames:
           0     1     2          3     4
15  5.076065  15.0  15.0   9.317888  15.0
16  5.740988  16.0  16.0  10.000000  16.0
17  6.405910  17.0  17.0  10.000000  17.0
18  7.070833  18.0  18.0  10.000000  18.0
19  7.735755  19.0  19.0  10.000000  19.0
